In [17]:
# Libraries
import json
import pymongo
import requests
from lxml import html
import re

In [18]:
# Load database credentials from json file
with open('dbconfig.json') as config_file:
    config = json.load(config_file)

username = config['username']
password = config['password']
db_url = config['db_url']

# database string
CNX_STR = f"mongodb+srv://{username}:{password}@{db_url}/?connectTimeoutMS=50000"
client = pymongo.MongoClient(CNX_STR)

# Check if the database exists
if "manage2sail" in client.list_database_names():
    print("Database exists!")
    db = client.manage2sail
else:
    print("Database does not exist!")

Database does not exist!


In [19]:
# scrape the data
base_url = 'https://www.manage2sail.com/en-US/search'
params = {
    'filterYear': '2023',
    'filterMonth': '',
    'filterCountry': 'SUI',
    #'filterRegion': 'e3165406-2424-470e-9ec2-d9dac6c9b742', # Zürichsee
    'filterRegion': '',
    'filterClass': '',
    'filterClubId': '',
    'filterScoring': '',
    'paged': 'false',
    'filterText': ''
}

all_data = []
response = requests.get(base_url, params=params)

if response.status_code == 200:
    # Parse the HTML content
    tree = html.fromstring(response.content)
    table = tree.xpath('/html/body/div[2]/div[3]/div[2]/div/table')[0]
    data = []

    for row in table.xpath('./tbody/tr'):
        row_data = [cell.text_content().strip() for cell in row.xpath('./td')]
        link = row.xpath('./td[4]/a/@href')
        row_data.append(link[0] if link else None)
        data.append(row_data)

    all_data.extend(data)
else:
    print(f"Failed to retrieve page")

# print(all_data)

formatted_data = [
    {
        'year': row[0],
        'startdate': row[1],
        'enddate': row[2],
        'nameevent': row[3],
        'status': row[4],
        'nation': row[5],
        'city': row[6],
        'hostclub': row[7],
        'link': row[8]
    } for row in all_data
]

# To visualize the formatted data
print(formatted_data[:5])  # Print the first 5 for brevity

[{'year': '2023', 'startdate': '01.01.', 'enddate': '08.01.', 'nameevent': '29er Wintertraining Camp III', 'status': 'Cancelled', 'nation': '', 'city': '', 'hostclub': 'Regionalverband Bodensee/Rhein', 'link': '/en-US/event/c5e5bda4-0140-42cc-ae8b-1cec508c5d95'}, {'year': '2023', 'startdate': '01.01.', 'enddate': '02.04.', 'nameevent': "Challenge d'Hiver 22-23", 'status': 'Finished', 'nation': 'SUI', 'city': 'Lausanne', 'hostclub': 'Cercle de la voile de Vidy', 'link': '/en-US/event/9358826e-1e64-488a-96cb-2ddeb5c544bc'}, {'year': '2023', 'startdate': '08.01.', 'enddate': '08.01.', 'nameevent': 'Rois', 'status': 'Finished', 'nation': 'SUI', 'city': 'Villeneuve', 'hostclub': 'Cercle de la voile de Villeneuve', 'link': '/en-US/event/gratuit'}, {'year': '2023', 'startdate': '13.01.', 'enddate': '15.01.', 'nameevent': 'ACVL TRAINING FOR -- 29er - Hyères (winter training #4)', 'status': 'Closed', 'nation': 'FRA', 'city': 'HYERES', 'hostclub': 'Association des Clubs de Voile Lémaniques', 'li

In [20]:
#Load data to db
db.events.insert_many(formatted_data)

InsertManyResult([ObjectId('6679e81d6454fb6fd4724b78'), ObjectId('6679e81d6454fb6fd4724b79'), ObjectId('6679e81d6454fb6fd4724b7a'), ObjectId('6679e81d6454fb6fd4724b7b'), ObjectId('6679e81d6454fb6fd4724b7c'), ObjectId('6679e81d6454fb6fd4724b7d'), ObjectId('6679e81d6454fb6fd4724b7e'), ObjectId('6679e81d6454fb6fd4724b7f'), ObjectId('6679e81d6454fb6fd4724b80'), ObjectId('6679e81d6454fb6fd4724b81'), ObjectId('6679e81d6454fb6fd4724b82'), ObjectId('6679e81d6454fb6fd4724b83'), ObjectId('6679e81d6454fb6fd4724b84'), ObjectId('6679e81d6454fb6fd4724b85'), ObjectId('6679e81d6454fb6fd4724b86'), ObjectId('6679e81d6454fb6fd4724b87'), ObjectId('6679e81d6454fb6fd4724b88'), ObjectId('6679e81d6454fb6fd4724b89'), ObjectId('6679e81d6454fb6fd4724b8a'), ObjectId('6679e81d6454fb6fd4724b8b'), ObjectId('6679e81d6454fb6fd4724b8c'), ObjectId('6679e81d6454fb6fd4724b8d'), ObjectId('6679e81d6454fb6fd4724b8e'), ObjectId('6679e81d6454fb6fd4724b8f'), ObjectId('6679e81d6454fb6fd4724b90'), ObjectId('6679e81d6454fb6fd4724b

In [21]:
# remove all events that are seminar (eventname contains "\nS$")
collection = db.events
filter_criteria = {"nameevent": { "$regex": "\nS$", "$options": "i" }}
result = collection.delete_many(filter_criteria)

deleted_count = result.deleted_count
print(f"{deleted_count} documents with nameevent containing '\s' (seminar)  have been deleted.")

47 documents with nameevent containing '\s' (seminar)  have been deleted.


In [22]:
# remove all events that contain "jeden" in their nameevent inidactin a weekliy recuring event
collection = db.events
pipeline = [
    {"$match": {"nameevent": {"$regex": "jeden", "$options": "i"}}},
    {"$group": {"_id": None, "ids": {"$push": "$_id"}}}
]

result = collection.aggregate(pipeline)
deleted_count = 0
for doc in result:
    if "ids" in doc:
        deleted_count = len(doc["ids"])
        collection.delete_many({"_id": {"$in": doc["ids"]}})

print(f"{deleted_count} documents with eventname containing 'jeden' have been deleted.")

2 documents with eventname containing 'jeden' have been deleted.


In [23]:
# delete all events with status "Cancelled" (deleted events)
collection = db.events
pipeline = [
    {"$match": {"$and": [
        {"status": "Cancelled"}
    ]}},
    {"$group": {"_id": None, "ids": {"$push": "$_id"}}}
]

result = collection.aggregate(pipeline)
deleted_count = 0
for doc in result:
    if "ids" in doc:
        deleted_count = len(doc["ids"])
        collection.delete_many({"_id": {"$in": doc["ids"]}})

print(f"{deleted_count} documents with status 'Cancelled' have been deleted.")

15 documents with status 'Cancelled' have been deleted.


In [24]:
# clean hostclub column 
def clean_spaces(text):
    # remove " Zürichsee-Segler-Verband" if present
    text = text.replace("Zürichsee-Segler-Verband", "")
    # replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # remove leading and trailing spaces
    text = text.strip()
    return text

collection = db.events
updated_count = 0  

for document in collection.find():
    if 'hostclub' in document:
        # Clean the hostclub field
        cleaned_hostclub = clean_spaces(document['hostclub'])
        # Update the document in the database
        update_result = collection.update_one({'_id': document['_id']}, {'$set': {'hostclub': cleaned_hostclub}})
        if update_result.modified_count > 0:
            updated_count += 1

print(f"{updated_count} documents have been updated - hostclub column cleaned.")

60 documents have been updated - hostclub column cleaned.


In [25]:
# reformat the date columns
import datetime # otherwise throws an error
collection = db.events
updated_count = 0

for document in collection.find():
    # check if fields are present
    if 'year' in document and 'startdate' in document and 'enddate' in document:
        # Construct full date strings
        full_startdate_str = document['startdate'] + " " + str(int(document['year']))
        full_enddate_str = document['enddate'] + " " + str(int(document['year']))
        
        # convert 
        startdate = datetime.datetime.strptime(full_startdate_str, "%d.%m. %Y")
        enddate = datetime.datetime.strptime(full_enddate_str, "%d.%m. %Y")
        
        # update the document in the database with the formatted dates
        update_result = collection.update_one(
            {'_id': document['_id']},
            {'$set': {
                'iso_startdate': startdate.isoformat(),
                'iso_enddate': enddate.isoformat(),
                'startdate': startdate,
                'enddate': enddate
            }}
        )
        
        if update_result.modified_count > 0:
            updated_count += 1

print(f"{updated_count} documents have been formatted and updated.")

427 documents have been formatted and updated.


In [26]:
# remove column 'year' from all documents
collection = db.events

updated_count = 0
for document in collection.find():
    # check if 'year' field is present
    if 'year' in document:
        # update document
        update_result = collection.update_one(
            {'_id': document['_id']},
            {'$unset': {'year': ''}}
        )

        if update_result.modified_count > 0:
            updated_count += 1

print(f"{updated_count} documents have been updates to remove ")

427 documents have been updates to remove 


In [27]:
# add prefix to all link, so it is complete
collection = db.events
updated_count = 0

for document in collection.find():
    # check if the document has a 'link' field
    if 'link' in document:
        # "https://manage2sail.com" as prefix of each link
        updated_link = "https://manage2sail.com" + document['link']
        
        # update the document
        update_result = collection.update_one(
            {'_id': document['_id']},
            {'$set': {'link': updated_link}}
        )
        
        if update_result.modified_count > 0:
            updated_count += 1

print(f"{updated_count} documents have been updated with the link prefix.")

427 documents have been updated with the link prefix.
